# <font color='red'>STOCK MARKET IS VERY RISKY UNTIL YOU DO IT PROPERLY. PLEASE DO NOT TAKE TRADES JUST BECAUSE THIS TOOL GIVES YOU THE NAME. APPLY YOUR OWN LEARNINGS, CREATE YOUR OWN STRATEGY, ASSESS RISK & TRADE THE PLAN.</font>

In [1]:
'065676'

'065676'

# Imports

In [2]:
import backtrader
from helpers.datahandler import *
from helpers.investing import *
from helpers.journal_handler import *
from helpers.intraday import *
from helpers.nse_data import *

# Defaults

In [4]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

In = Investing(check_fresh = True) # Investing or Swing trading analysis
Intra = IntraDay() # Intraday strategies and analysis
NSE = NSEData() # Get NSE Live data
MS = MarketSentiment() # Get Live Market Sentiment TICK, TRIN and 52W high/ low stocks

Checking Fresh Data.....


/home/admin1/Desktop/ML Study/Finance/helpers/datahandler.py:49: UserWarning: No CSV data files present at ./data Downloading new data for analysis
  warnings.warn(f"No CSV data files present at {self.data_path} Downloading new data for analysis")


# Journal
Make a journal in Google Drive for each trade you take to analyse. It is a not so simple process and need to connect with Google API so if you're not comfortable with it, just skip it

In [3]:
private_key = join(expanduser('~'),'Documents','client_secret.json') # Personal for each user. Mine is stored in /home/Documents
JH = JournalHandler(private_key)

journal = JH.get_journal('Finance Journal') # My Google Sheet jornam has name "Finance Journal"

JH.total_pl(journal) # total Profit and Loss up until now

--------------------------------------------------------------------------- 

Trailing Stop Loss: 

DABUR bought on: 2021-09-01 00:00:00,

--------------------------------------------------------------------------- 



/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


502.65

# <font color='teal'>Swing Strategies </font>
## All Details
Basic Details about stocks such as Momentum, Ichimoku, 1-2-3 candles pattern etc

In [3]:
budget = 40000

df = In.get_recent_info(nifty = 100, **{'mvs':[20,50,100,200]})
df = df[(df['Overbought'] == False) & (df['Ichi Count'] > 0) & 
        (df['Over 50-SMA'] == True) & (df['Over 100-SMA'] == True) & (df['Over 200-SMA'] == True) &
       (df['Momentum ADX'] > 25) & 
        (df['LTP'] <= budget)]

df.head(2)

/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,Date,Name,LTP,Index,Over 20-SMA,Over 50-SMA,Over 100-SMA,Over 200-SMA,Overbought,Oversold,Momentum ADX,Ichi Count,1 Candle,2 Candles,3 Candles
1,2021-09-24,ABBOTINDIA,21204.95,Nifty 100,True,True,True,True,False,False,40.207099,2,Unknown Red,Bearish Engulfing,Unknown
6,2021-09-24,ALKEM,4003.10,Nifty 100,True,True,True,True,False,False,34.607216,2,Unknown Green,Unknown,Unknown


## 44-SMA 
If you find a `Green` candle taking support on 44 days `Simple Moving Average` line and stock is in uptrend, you can take the trade and set Risk to Reward ratio as 1:2

In [ ]:
budget = 40000 # Total Budget
risk = 400 # Risk per trade
df = In.show_full_stats(budget=budget,risk=risk, diff = 151, nifty='nifty_200') # Show only stocks undder your budget and  whose recent candle's lowest is not more than 151 rupees above the moving average line
df = df[(df['Expected Change %'] <= 15) & (df['RSI']<70) & (df['Direction'] != 'Falling Low')]
df

## Golden Crossover
When 44 days `SMA` Crosses 200 days `SMA` from below and you find a `Green` candle within 15 days taking support on `44` days line set Risk to Reward ratio as 1:5

In [7]:
for name in In.data['nifty_500']:
    df = In.open_downloaded_stock(name)
    if In.has_golden_crossover(df,lookback=10):
        print(name)

ESCORTS
ITC


## Oversold / Overbought Stocks
Buy when RSI is less than 30 sell when greater than 70

In [8]:
for name in In.data['nifty_100']:
    df = In.open_downloaded_stock(name)
    if In.get_RSI(df) < 30:
        print(name)

## MACD Signal
It is the strategy when MACD (Blue) Line cuts the Signal (Red) line from below then it is a buy signal. When the blue line cuts from upside down, it is sell signal

In [4]:
for name in In.data['nifty_100']:
    df = In.open_downloaded_stock(name)
    if In.macd_signal(df) == 'Buy':
        print(name)

ASIANPAINT
BERGEPAINT
HDFCBANK
MRF


# <font color='teal'>Intraday Strategies</font>

## Narrow Range 7
If recent candle's range is lowest among 6 other days then the future candle might break it's low/high

In [9]:
for name in In.data['nifty_100']:
    if Intra.NR7_strategy(name):
        print(name)

ABBOTINDIA
ADANIPORTS
ASIANPAINT
AUROPHARMA
EICHERMOT
GODREJCP
HINDPETRO
IGL
INDUSINDBK
NAUKRI
INFY
SHREECEM
TATACONSUM
TECHM
UPL
VEDL
WIPRO


## Whole Number Open
When Open == Low / High in Whole number as `xxx.00` within 15 minutes

In [9]:
result = Intra.whole_number_strategy(nifty=200, min_val=101,max_val=4999)
for key in result.keys():
    print(key,":\n")
    for r in sorted(result[key],key=lambda x: int(x[-1].split(' ')[-1])):
        print(r,'\n')

Long :

('HEROMOTOCO', '1.88%', 'Nifty 50') 

('ADANIENT', '1.56%', 'Nifty 100') 

('ATGL', '2.89%', 'Nifty 200') 

Short :

('HINDUNILVR', '-1.19%', 'Nifty 50') 

('GLAND', '-2.0%', 'Nifty 100') 

('ADANIGREEN', '-2.59%', 'Nifty 100') 

('HAVELLS', '-1.77%', 'Nifty 100') 

('IGL', '-2.04%', 'Nifty 100') 

('POLYCAB', '-1.24%', 'Nifty 200') 

('AUBANK', '-1.76%', 'Nifty 200') 

('GLENMARK', '-1.96%', 'Nifty 200') 

('VOLTAS', '-2.86%', 'Nifty 200') 



## `%` Change probability
Gives the stock names which have the highest probability of providing you `x%` in intraday atleast on both long or short side based on the previous `N` days record

In [7]:
# Gives you top 10 stocks from Nifty 500 which have the highest probability of giving atleast 0.99% on your investment based on the Long/Buy position. Data is based on past 60 trading days
# Test it for different time duration, index, % change and Long / Short criteria based on overall market trend

Intra.prob_by_percent_change(symbol = None, index = 500, time_period = 60, change_percent=0.099, sort_by='Long Probability', top_k = 10) 

{'IDEA': {'Long Probability': 0.85, 'Short Probability': 0.77},
 'LINDEINDIA': {'Long Probability': 0.83, 'Short Probability': 0.67},
 'SUNTECK': {'Long Probability': 0.82, 'Short Probability': 0.65},
 'BRIGADE': {'Long Probability': 0.78, 'Short Probability': 0.73},
 'GUJALKALI': {'Long Probability': 0.78, 'Short Probability': 0.45},
 'HFCL': {'Long Probability': 0.78, 'Short Probability': 0.68},
 'IBREALEST': {'Long Probability': 0.78, 'Short Probability': 0.77},
 'PNCINFRA': {'Long Probability': 0.78, 'Short Probability': 0.6},
 'ADANITRANS': {'Long Probability': 0.77, 'Short Probability': 0.67},
 'BAJAJELEC': {'Long Probability': 0.77, 'Short Probability': 0.67}}

# <font color='Teal'>Live Data</font>


## Market Sentiment: TICK, TRIN (Arm's Index) & 52W high-Low touching shares


In [10]:
MS.get_live_sentiment()

{'Latest Updated on': '27/09/2021 10:54:59',
 'Up': 691,
 'Down': 805,
 'TICK': -114,
 'Volume Up': 444.1,
 'Volume Down': 417.42,
 'TRIN': 0.806,
 '52W High': 54,
 '52W Low': 12}

## Indices Based
Buy or Sell stock based on which indices are performing highest in first 5 mins. Buy or sell stocks only for those indices which arevperforming the greatest

In [16]:
NSE.current_indices_status(10) # Live as of market is open. works post market also

,index,percentChange,key,variation
8,INDIA VIX,4.15,BROAD MARKET INDICES,0.70
20,NIFTY IT,-2.44,SECTORAL INDICES,-906.90
26,NIFTY REALTY,1.69,SECTORAL INDICES,8.45
16,NIFTY AUTO,1.69,SECTORAL INDICES,176.10
27,NIFTY HEALTHCARE INDEX,-1.53,SECTORAL INDICES,-137.35
43,NIFTY200 QUALITY 30,-1.23,STRATEGY INDICES,-190.75
23,NIFTY PHARMA,-1.23,SECTORAL INDICES,-176.25
44,NIFTY ALPHA LOW-VOLATILITY 30,-1.23,STRATEGY INDICES,-223.85
24,NIFTY PSU BANK,1.21,SECTORAL INDICES,28.50
39,NIFTY ALPHA 50,-1.02,STRATEGY INDICES,-323.95


In [15]:
NSE.open_nse_index('NIFTY IT')

,symbol,pChange,open,dayHigh,dayLow,lastPrice,Index
10,COFORGE,-4.78,5690.00,5695.10,5330.20,5378.85,Nifty 200
9,MPHASIS,-4.10,3399.95,3399.95,3190.50,3205.45,Nifty 200
8,MINDTREE,-4.07,4640.00,4641.80,4380.00,4413.95,Nifty 200
7,LTI,-3.71,6300.00,6368.40,5930.00,6000.00,Nifty 100
6,TECHM,-3.15,1523.00,1525.00,1459.20,1467.00,Nifty 50
5,WIPRO,-2.95,678.05,678.35,651.00,656.55,Nifty 50
4,HCLTECH,-2.68,1366.00,1367.95,1314.25,1321.85,Nifty 50
0,NIFTY IT,-2.41,37335.90,37345.05,35986.20,36208.15,Other
3,OFSS,-2.37,4714.95,4739.90,4549.10,4598.90,Nifty 500
2,INFY,-1.70,1768.75,1769.00,1721.35,1733.95,Nifty 50


# <font color='teal'>Risk Management</font>

In [5]:
# You have a total budget of 40K, and a risk apetite of rs 400 per trade
# You can set manual such as # entry = 3980.1, stop_loss=3780.5

In.get_particulars('NIITLTD',40000,400,entry = 368.5, stop_loss=319)

{'Buying Price': 368.5,
 'Stop-Loss %': 13.43,
 'Target %': 26.85,
 'Quantity': 8.0,
 'Stop-Loss Price': 319,
 'Trigger Price': 467.45,
 'Risk Per Share': 49.5,
 'Profit Per Share': 98.95,
 'Max loss on this config': 396.0,
 'Max Gain on this config': 791.6,
 'Price To Profit Ratio': 3.72,
 'Index': 'Other'}